# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
#Импорт инструментов
import pandas as pd
import numpy as np

#Модули, связанные с ML
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from joblib import dump
from joblib import load
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Данные предобработки не требуют: все было уже сделано на предыдущих этапах.

Всего 3214 значений, 5 колонок:

Точнее, другим языком... Перед нами обучающий набор данных, где:

- 3214 объекта

- по столбикам: 5 столбцов, часть из которых признаки и один целевой признак

## Разбейте данные на выборки

Необходимо разделить исходные данные на обучающую, валидационную и тестовую выборки.

In [5]:
features = df.drop('is_ultra', axis=1)  
target = df['is_ultra'] 
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state = 12345, stratify=target)
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state = 12345, stratify=target)

In [6]:
from sklearn.model_selection import KFold 

<b># МОДЕЛЬ ДЕРЕВО РЕШЕНИЙ</b> - тройная кросс-валидация

In [7]:
X = features
y = target

k = 3 # 3 ВЫБОРКИ
kf = KFold(n_splits=k, random_state=12345)
 
acc_score = []

dtr = DecisionTreeRegressor(random_state=12345)
for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
     
    dtr.fit(X_train,y_train)
    pred_values = dtr.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
result = mean_squared_error(y_test, pred_values)**0.5
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

print("RMSE  without optimization:", result)

accuracy of each fold - [0.7313432835820896, 0.7236227824463118, 0.7133520074696545]
Avg accuracy : 0.7227726911660186
RMSE  without optimization: 0.535395174175436


<b># МОДЕЛЬ СЛУЧАЙНЫЙ ЛЕС</b> - тройная кросс-валидация

In [8]:
X = features
y = target

k = 3 # 3 ВЫБОРКИ
kf = KFold(n_splits=k, random_state=12345)
 
acc_score = []

rfc = RandomForestClassifier(random_state=12345, n_estimators=3)
for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
     
    rfc.fit(X_train,y_train)
    pred_values = rfc.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
result = mean_squared_error(y_test, pred_values)**0.5
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

print("RMSE  without optimization:", result)

accuracy of each fold - [0.746268656716418, 0.7450980392156863, 0.7675070028011205]
Avg accuracy : 0.7529578995777415
RMSE  without optimization: 0.4821752764284784


<b># МОДЕЛЬ ЛОГИЧЕСКАЯ РЕГРЕССИЯ</b> - тройная кросс-валидация

In [9]:
X = features
y = target

k = 3 # 3 ВЫБОРКИ
kf = KFold(n_splits=k, random_state=12345)
 
acc_score = []

lr = LogisticRegression(random_state=12345)
for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
     
    lr.fit(X_train,y_train)
    pred_values = lr.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
result = mean_squared_error(y_test, pred_values)**0.5
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

print("RMSE  without optimization:", result)

accuracy of each fold - [0.6921641791044776, 0.7161531279178338, 0.7329598506069094]
Avg accuracy : 0.7137590525430736
RMSE  without optimization: 0.5167592760590666


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


<b>НИЖЕ БУДЕТ СНОСОЧКА С ПРОВЕРКАМИ НА ТЕСТОВЫХ ВЫБОРКАХ И ОПТИМИЗАЦИИ !!! СМОТРЕТЬ ПО СНОСОЧКЕ ДАЛЕЕ !!!</b>

Масштабирование данных.

In [10]:
ss = StandardScaler()
features_train_scaled = ss.fit_transform(features_train)
features_test_scaled = ss.transform(features_test)
target_train = np.array(target_train)

Масштабирование выполняется из-за того, что разные величины выражены в разных единицах измерения. Эта процедура позволяет организовать «честную схватку» между признаками при определении их важности. Кроме того, мы конвертируем y_train из типа данных Pandas Series в массив NumPy для того чтобы позже модель смогла бы работать с соответствующими целевыми показателями.

Но, не знаю, насколько нам это надо в нашем кейсе...

## Исследуйте модели

В каждой из трех моделей будет блок кода без оптимизации гиперпараметров и с ней.

<b>Модель "Дерево решений"</b>

In [11]:
dtr = DecisionTreeRegressor(random_state=12345)
dtr.fit(features_train, target_train)
predictions_test = dtr.predict(features_test)
result = mean_squared_error(target_test, predictions_test)**0.5
accuracy = accuracy_score(target_test, predictions_test.round())
print("RMSE без оптимизации гиперпараметров:", result)
print("Доля правильных ответов без оптимизации гиперпараметров:", accuracy)

RMSE без оптимизации гиперпараметров: 0.5061807043661748
Доля правильных ответов без оптимизации гиперпараметров: 0.7437810945273632


In [12]:
best_model = None
best_result = 10000
best_depth = 0
for depth in range(1, 6):
    dtr = DecisionTreeRegressor(random_state=12345, max_depth=depth)
    dtr.fit(features_train, target_train)
    predictions_test = dtr.predict(features_test)
    result = mean_squared_error(target_test, predictions_test)**0.5
    accuracy = accuracy_score(target_test, predictions_test.round())
    if result < best_result:
        best_model = dtr
        best_result = result
        best_depth = depth
        best_accuracy = accuracy
        
print("RMSE наилучшей модели на валидационной выборке:", best_result, "Глубина дерева:", best_depth, "Доля правильных ответов:", best_accuracy)

RMSE наилучшей модели на валидационной выборке: 0.4067988188892464 Глубина дерева: 5 Доля правильных ответов: 0.7860696517412935


<b>Итого:</b> Доля правильных ответов: 0.786 > 0.75 !!!

<b>Модель "Случайный лес"</b>

In [13]:
rfc = RandomForestClassifier(random_state=12345, n_estimators=3)
rfc.fit(features_train, target_train)
predictions_test = rfc.predict(features_test)
result = mean_squared_error(target_test, predictions_test)**0.5
accuracy = accuracy_score(target_test, predictions_test.round())
print("RMSE без оптимизации гиперпараметров:", result)
print("Доля правильных ответов без оптимизации гиперпараметров:", accuracy)

RMSE без оптимизации гиперпараметров: 0.5012422379394292
Доля правильных ответов без оптимизации гиперпараметров: 0.7487562189054726


In [14]:
best_model = None
best_result = 10000
best_est = 0
best_depth = 0
for est in range(1, 51, 2):
    for depth in range (1, 11):
        rfc = RandomForestRegressor(random_state=12345, n_estimators=est, max_depth=depth)
        rfc.fit(features_train, target_train)
        predictions_test = rfc.predict(features_test)
        result = mean_squared_error(target_test, predictions_test)**0.5
        accuracy = accuracy_score(target_test, predictions_test.round())
        if result < best_result:
            best_model = rfc
            best_result = result
            best_est = est
            best_depth = depth
            best_accuracy = accuracy

print("RMSE наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth, "Доля правильных ответов:", best_accuracy)

RMSE наилучшей модели на валидационной выборке: 0.3768710106510978 Количество деревьев: 15 Максимальная глубина: 10 Доля правильных ответов: 0.8059701492537313


<b>Итого:</b> Доля правильных ответов: 0.806 > 0.75 !!!

<b>Модель "Логическая регрессия"</b>

In [15]:
lr = LogisticRegression(random_state=12345)
lr.fit(features_train, target_train)
predictions_test = lr.predict(features_test)
result = mean_squared_error(target_test, predictions_test)**0.5
accuracy = accuracy_score(target_test, predictions_test.round())
print("RMSE без оптимизации гиперпараметров:", result)
print("Доля правильных ответов без оптимизации гиперпараметров:", accuracy)

RMSE без оптимизации гиперпараметров: 0.5371752171594928
Доля правильных ответов без оптимизации гиперпараметров: 0.7114427860696517


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Из-за малого количества параметров линейная регрессия менее склонна к переобучению, чем, например, деревья решений.

Поэтому, как я понял, оптимизацию гиперпараметров мы тут провести не можем: изменить глубину и количество деревьев в "звене", ну или этому мы научимся попозже...

## Проверьте модель на тестовой выборке

По сути, я это уже сделал выше...

<b>СНОСОЧКА!!!</b>

In [22]:
b = list()
c = list()
d = list()

<b># МОДЕЛЬ ДЕРЕВО РЕШЕНИЙ</b>

In [23]:
X = features
y = target

k = 3 # 3 ВЫБОРКИ
kf = KFold(n_splits=k, random_state=12345)
 
acc_score = []

dtr = DecisionTreeRegressor(random_state=12345)
for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]

best_model = None
best_result = 10000
best_depth = 0
for depth in range(1, 6):
    dtr = DecisionTreeRegressor(random_state=12345, max_depth=depth)
    dtr.fit(X_train, y_train)
    predictions_test = dtr.predict(X_test)
    result = mean_squared_error(y_test, predictions_test)**0.5
    accuracy = accuracy_score(y_test, predictions_test.round())
    if result < best_result:
        best_model = dtr
        best_result = result
        best_depth = depth
        best_accuracy = accuracy
       
print("RMSE наилучшей модели на валидационной выборке:", best_result, "Глубина дерева:", best_depth, "Доля правильных ответов:", best_accuracy)
b.append(best_result)
c.append(f'Глубина дерева: {best_depth}')
d.append(best_accuracy)

RMSE наилучшей модели на валидационной выборке: 0.3881586616312411 Глубина дерева: 5 Доля правильных ответов: 0.7917833800186741


<b># МОДЕЛЬ СЛУЧАЙНЫЙ ЛЕС</b>

In [24]:
X = features
y = target

k = 3 # 3 ВЫБОРКИ
kf = KFold(n_splits=k, random_state=12345)
 
acc_score = []

rfc = RandomForestClassifier(random_state=12345)
for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]

best_model = None
best_result = 10000
best_est = 0
best_depth = 0
for est in range(1, 51, 2):
    for depth in range (1, 11):
        rfc = RandomForestRegressor(random_state=12345, n_estimators=est, max_depth=depth)
        rfc.fit(X_train, y_train)
        predictions_test = rfc.predict(X_test)
        result = mean_squared_error(y_test, predictions_test)**0.5
        accuracy = accuracy_score(y_test, predictions_test.round())
        if result < best_result:
            best_model = rfc
            best_result = result
            best_est = est
            best_depth = depth
            best_accuracy = accuracy

print("RMSE наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth, "Доля правильных ответов:", best_accuracy)
b.append(best_result)
c.append(f'Количество деревьев: {best_est}')
c.append(f'Максимальная глубина: {depth}')
d.append(best_accuracy)

RMSE наилучшей модели на валидационной выборке: 0.36832145512452796 Количество деревьев: 49 Максимальная глубина: 10 Доля правильных ответов: 0.8169934640522876


<b># МОДЕЛЬ ЛОГИЧЕСКАЯ РЕГРЕССИЯ</b>

In [25]:
X = features
y = target

k = 3 # 3 ВЫБОРКИ
kf = KFold(n_splits=k, random_state=12345)
 
acc_score = []

lr = LogisticRegression(random_state=12345)
for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
    
lr.fit(X_train, y_train)
predictions_test = lr.predict(X_test)
result = mean_squared_error(y_test, predictions_test)**0.5
accuracy = accuracy_score(y_test, predictions_test.round())
print("RMSE без оптимизации гиперпараметров:", result)
print("Доля правильных ответов без оптимизации гиперпараметров:", accuracy)
b.append('')
b.append(result)
c.append('Без гиперпараметров')
d.append('')
d.append(accuracy)

RMSE без оптимизации гиперпараметров: 0.5167592760590666
Доля правильных ответов без оптимизации гиперпараметров: 0.7329598506069094


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [26]:
a = ['МОДЕЛЬ ДЕРЕВО РЕШЕНИЙ', 'МОДЕЛЬ СЛУЧАЙНЫЙ ЛЕС', '', 'МОДЕЛЬ ЛОГИЧЕСКАЯ РЕГРЕССИЯ']

df_all = pd.DataFrame({"Название модели": a, "Наилучшее RMSE": b, "Оптимизированные гиперпараметры": c, "Наилучшая доля правильных ответов": d})

<b>ТАБЛИЦА</b> - упоротая конечно, но таблица

In [27]:
df_all

,Название модели,Наилучшее RMSE,Оптимизированные гиперпараметры,Наилучшая доля правильных ответов
0,МОДЕЛЬ ДЕРЕВО РЕШЕНИЙ,0.388159,Глубина дерева: 5,0.791783
1,МОДЕЛЬ СЛУЧАЙНЫЙ ЛЕС,0.368321,Количество деревьев: 49,0.816993
2,,,Максимальная глубина: 10,
3,МОДЕЛЬ ЛОГИЧЕСКАЯ РЕГРЕССИЯ,0.516759,Без гиперпараметров,0.73296


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
